<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/MACHINE%20LEARNING%20Y%20DEEP%20LEARNING/cnns_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST dataset

La base de datos MNIST (Modified National Institute of Standards and Technology) es un conjunto de datos ampliamente utilizado en la comunidad de aprendizaje automático para entrenar y probar modelos de clasificación de imágenes. Contiene imágenes de dígitos escritos a mano del 0 al 9, y es especialmente útil para trabajar con técnicas de procesamiento de imágenes y redes neuronales.

Características de la Base de Datos MNIST:

* **Número de Imágenes**: 70,000 imágenes en total.
* **Conjunto de Entrenamiento**: 60,000 imágenes.
* **Conjunto de Prueba**: 10,000 imágenes.
* **Dimensiones de las Imágenes**: Cada imagen es de 28x28 píxeles.
* **Formato de las Imágenes**: Escala de grises (un solo canal).
* **Etiquetas**: Cada imagen está etiquetada con el dígito correspondiente (0-9).

Estructura de las Imágenes:

* Pixeles: Los valores de los píxeles varían de 0 a 255.
* 0: Representa un píxel negro (fondo).
* 255: Representa un píxel blancos (trazo del dígito).

## Carga de la Base de Datos MNIST

PyTorch proporciona herramientas dentro de torchvision para cargar y preprocesar la base de datos MNIST fácilmente.

In [ ]:
import torch
import torchvision.transforms as T

from torchvision import datasets

## Data Augmentation

Podemos aplicar transformaciones a nuestros datos

In [ ]:
train_transform = T.Compose([
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor()])

valid_transform = T.Compose([
            T.RandomCrop(32, padding=4),
            T.ToTensor()])

In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = train_transform,
    download = True)

In [ ]:
valid_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = valid_transform,
    download = True)

In [ ]:
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = valid_transform)

In [ ]:
print("train size: {}".format(len(train_data)))
print("test size : {}".format(len(test_data)))

## Dataloader

El DataLoader en PyTorch es una herramienta fundamental para la preparación y el suministro de datos a modelos de aprendizaje automático. Facilita la carga, el procesamiento y la iteración eficiente de grandes conjuntos de datos durante el entrenamiento y la evaluación de modelos. Utiliza múltiples subprocesos para cargar datos en paralelo, acelerando la preparación de los datos y reduciendo los tiempos de espera durante el entrenamiento


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

## Separa los datos que no son del **test** en dos partes

* Datos de entrenamiento
* Datos de validación

In [ ]:
import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler

def split_train_valid_data(
    train_data,
    valid_data,
    valid_size=0.1,
    batch_size=32,
    shuffle=True,
    random_seed=0,
    num_workers=4):

  num_train = len(train_data)
  indices = list(range(num_train))
  split = int(np.floor(valid_size * num_train))

  if shuffle == True:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx, valid_idx = indices[split:], indices[:split]

  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = torch.utils.data.DataLoader(train_data,
                    batch_size=batch_size, sampler=train_sampler,
                    num_workers=num_workers)

  valid_loader = torch.utils.data.DataLoader(valid_data,
                    batch_size=batch_size, sampler=valid_sampler,
                    num_workers=num_workers)

  return (train_loader, valid_loader)

#Visualización

Podemos utilizar la librería **matplotlib** para visualizar las imágenes de la base de datos MNIST

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(train_data.data[0], cmap='gray')
plt.title('%i' % train_data.targets[0])
plt.axis("off")
plt.show()

In [ ]:
figure = plt.figure(figsize=(8, 9))
cols, rows = 5, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

# Modelo LeNet-5
LeNet-5 es uno de los primeros modelos de redes neuronales convolucionales (CNNs) y fue desarrollado por científicos---entre ellos Yann LeCun---en 1998. Este modelo se diseñó principalmente para reconocer dígitos escritos a mano y fue utilizado para la lectura automatizada de cheques y documentos. LeNet-5 sentó las bases para el desarrollo de CNNs más avanzadas y modernas, influyendo significativamente en el campo del reconocimiento de patrones y el aprendizaje profundo.

### 1. Arquitectura del Modelo LeNet-5

LeNet-5 tiene una arquitectura sencilla y bien estructurada, que consta de 7 capas (excluyendo las capas de entrada y salida). Estas capas incluyen tres capas convolucionales, dos capas de subsampling (o pooling), y dos capas completamente conectadas.

### 1.1. Capa de Entrada:

* Dimensión: 32x32 píxeles (imagen en escala de grises).
* Nota: Las imágenes de MNIST originalmente son de 28x28 píxeles, por lo que se * agregan bordes para aumentar a 32x32 píxeles.

### 1.2. Capa Convolucional **conv1**:

* Filtros: 6 filtros de 5x5.
* Salida: 6 mapas de características de 28x28 (32 - 5 + 1 = 28).

### 1.3. Capa de Subsampling **s1**:

* Operación: Subsampling (promedio) con una ventana de 2x2.
* Salida: 6 mapas de características de 14x14.

### 1.4. Capa Convolucional **conv2**:

* Filtros: 16 filtros de 5x5.
* Salida: 16 mapas de características de 10x10 (14 - 5 + 1 = 10).

### 1.5. Capa de Subsampling **s2**:

* Operación: Subsampling (promedio) con una ventana de 2x2.
* Salida: 16 mapas de características de 5x5.

### 1.6. Capa Convolucional **conv3**:

Filtros: 120 filtros de 5x5.
Salida: 120 mapas de características de 1x1 (5 - 5 + 1 = 1).

### 1.7. Capa Completamente Conectada **f1**:

* Neuronas: 84.
* Operación: Función de activación sigmoid o tanh.

### 1.8 Capa Completamente Conectada **f2** (salida):

* Neuronal: 10 (una por cada dígito del 0 al 9).
* Operación: Función de activación **softmax** para clasificación.

### 1.2. Flujo de Datos en la Red
* Entrada: Imagen de 32x32 píxeles.
* **conv1**: Convolución → 28x28x6.
* **s2**: Subsampling → 14x14x6.
* **conv2**: Convolución → 10x10x16.
* **s2**: Subsampling → 5x5x16.
* **conv3**: Convolución → 1x1x120.
* **f1**: Conexión completa → 84.
* **f2**: Conexión completa → 10 (clases).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # conv1: entrada 1 canal, salida 6 canales, tamaño de kernel 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)

        # conv1: entrada 6 canales, salida 16 canales, tamaño de kernel 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)

        # conv3: entrada 16*4*4, salida 120
        self.conv3 = nn.Conv2d(16, 120, 5)

        # f2: entrada 120, salida 84
        self.fc1 = nn.Linear(120, 84)
        # Capa completamente conectada 3: entrada 84, salida 10 (dígitos)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        # Aplicar la primera capa convolucional seguida de ReLU y max pooling
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, 2)

        # Aplicar la segunda capa convolucional seguida de ReLU y max pooling
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, 2)

        # Aplicar la tercera capa convolucional seguida de ReLU
        x = F.relu(self.conv3(x))

        # Aplanar los datos para la capa completamente conectada
        x = x.view(-1, 120)

        # Aplicar la primera capa completamente conectada seguida de ReLU
        x = F.relu(self.fc1(x))

        # Aplicar la segunda capa completamente conectada
        x = self.fc2(x)

        # No es necesario aplicar el softmax si se utiliza el "CrossEntropyLoss"
        #x = F.softmax(x, dim=1)
        return x

Luego, debemos instanciar el modelo

In [ ]:
model = LeNet()

print(model)

Podemos probar para un salida cualquier que tipo de salida obtenemos.

In [ ]:
x = torch.rand(32,1,32,32)

# 'x' es un 'batch' de cuatro imagenes de 28x28x1
y = model(x)

print(y.shape)

## La dimension de los 'kernels' de nuestro modelo

Capas convolucionales

In [ ]:
print(f"conv1 shape (W): {model.conv1.weight.size()}")
print(f"conv1 shape (b): {model.conv1.bias.size()}")

print(f"conv2 shape (W): {model.conv2.weight.size()}")
print(f"conv2 shape (b): {model.conv2.bias.size()}")

print(f"conv3 shape (W): {model.conv3.weight.size()}")
print(f"conv3 shape (b): {model.conv3.bias.size()}")

Capas 'Full Connected'

In [ ]:
print(f"fc shape (W): {model.fc1.weight.size()}")
print(f"fc1 shape (b): {model.fc1.bias.size()}")

print(f"fc2 shape (W): {model.fc2.weight.size()}")
print(f"fc2 shape (b): {model.fc2.bias.size()}")

## Función de Activación **Softmax**

La función softmax es una función de activación que convierte un vector de valores arbitrarios en un vector de probabilidades, donde la suma de todas las probabilidades es igual a 1. Es especialmente útil en problemas de clasificación donde se necesita asignar probabilidades a diferentes clases (múltiples clases).

Matemáticamente, la función softmax para una entrada $\mathbf{z}=[z_1,z_2,\cdots,z_N]$:

$$y_i=\texttt{softmax}(z_i)=\frac{e^{z_i}}{\sum_{k=1}^N e^{z_k}}$$

donde $e$ es la base de logaritmo natural.

**Aplicación de Softmax en Tareas de Clasificación:**

* **Transformación a Probabilidades**: Esto es crucial para interpretar las salidas del modelo como probabilidades de pertenencia a cada clase.

* **Facilita la Comparación entre Clases**: La clase con la probabilidad más alta después de aplicar softmax es la predicción del modelo.

* **Compatibilidad con la Función de Pérdida**: La función de **Costo de Entropía Cruzada (Cross-Entropy)**, comúnmente utilizada en tareas de clasificación, requiere que las salidas del modelo sean probabilidades. Softmax garantiza que las salidas del modelo sean compatibles con esta función de pérdida.

# Función de Costo

La función de costo o pérdida de entropía cruzada (Cross-Entropy Loss) es ampliamente utilizada en tareas de clasificación. Esta función mide la diferencia entre las distribuciones de probabilidad predichas por el modelo y las verdaderas etiquetas de clase.

## Definición Matemática

Para un solo ejemplo de entrada con $N$ clases, la entropía cruzada se define como:

 $$CE=-\frac{1}{\mathcal{B}}\sum_{i}^{\mathcal{B}}t_i\log(y_i)$$

donde $\mathcal{B}$ es el tamaño del batch, es decir la cantidad de imagenes que se está utilizando en cada iteración. Además:

* $t_i$ es el valor verdadero de la $i$-ésima clase (0 o 1, en **one-hot encoding**).
* $y_i$ es la probabilidad predicha por el modelo para la $i$-ésima clase.

In [ ]:
loss_function = nn.CrossEntropyLoss()

# ¿Que es un optimizador?

Un optimizador es un algoritmo que ajusta iterativamente los parámetros (pesos y biases) de un modelo de red neuronal para minimizar la función de pérdida durante el entrenamiento. El objetivo principal del optimizador es encontrar los valores de los parámetros que hacen que el modelo realice una predicción de los resultados más cercana a los valores reales en los datos de entrenamiento.

**¿Cómo Funciona un Optimizador?**

* **Inicialización de Parámetros**: Los parámetros del modelo se inicializan con valores aleatorios.

* **Cálculo de la Pérdida**: La función de pérdida se calcula para medir cuán lejos están las predicciones del modelo de las etiquetas reales.

* **Cálculo del Gradiente**: Se utiliza el algoritmo de *Backpropagation* para calcular los gradientes de la función de pérdida con respecto a cada parámetro del modelo.

* **Actualización de Parámetros**: Los parámetros del modelo se **actualizan en la dirección opuesta a la del gradiente para reducir la pérdida**. La magnitud de cada actualización está controlada por la **tasa de aprendizaje** o *Learning Rate*.

Este ciclo se repite durante múltiples épocas hasta que la función de pérdida se minimiza o se alcanza un criterio de parada.

# El optimizador Adam

1. $m_t\leftarrow \beta_1\cdot m_{t-1}+(1-\beta_1)\cdot g_t$
2. $v_t\leftarrow \beta_2\cdot v_{t-1}+(1-\beta_2)\cdot g^2_t$
3. $\hat{m}_t\leftarrow \frac{m_t}{1-\beta^t_1}$
4. $\hat{v}_t\leftarrow \frac{v_t}{1-\beta^t_2}$
5. $\theta_t\leftarrow \theta_{t-1}-\alpha\cdot \frac{\hat{m}_t}{\sqrt{\hat{v}_t}+\epsilon}$

donde:

* $\theta_t$: El parámetro del modelo en la iteración $t$.
* $\alpha$: La tasa de aprendizaje
* $\epsilon$: Es un pequeño valor constante para evitar la división por cero (típicamente $10^{-8}$).
* $m_t$: El momento de primer orden.
* $v_t$: El momento de segundo orden.
* $g_t$: La gradiente en la iteración $t$.


## Beneficios del Optimizador Adam

* **Rápida Convergencia**: Adam tiende a converger más rápidamente que otros optimizadores, lo cual es particularmente útil en redes neuronales profundas y grandes conjuntos de datos.

* **Robustez**: Adam es robusto frente a hiperparámetros mal ajustados, lo que lo hace más fácil de usar en comparación con otros optimizadores que requieren una cuidadosa sintonización de hiperparámetros.

* **Estabilidad**: La combinación de momentos de primer y segundo orden proporciona una actualización más estable, lo que ayuda a evitar oscilaciones en el proceso de entrenamiento.

* **Eficiencia Computacional**: Aunque Adam realiza cálculos adicionales en comparación con **SGD**, sigue siendo computacionalmente eficiente .

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Entrenamiento

El entrenamiento nos permite ajustar los parámetros del modelo para minimizar la función de pérdida en los datos de entrenamiento.

Durante el entrenamiento, el modelo aprende a mapear las entradas a las salidas correctas a través de un proceso iterativo. Se utiliza el conjunto de datos de entrenamiento para calcular el la función de pérdida, los gradientes y luego actualizar los parámetros del modelo.

In [ ]:
def train(model, train_data, valid_data, optimizer, num_epochs):
  train_loader, valid_loader = split_train_valid_data(train_data, valid_data)

  model.train()

  train_steps = len(train_loader) # nro. de batch

  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      output = model(images)
      loss   = loss_function(output, labels)

      # Limpiar gradientes
      optimizer.zero_grad()

      # Backpropagation: Calculo de la gradientes
      loss.backward()

      # Actualizando de los parametros
      optimizer.step()

      if (i+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], it: [{i+1:0>4}/{train_steps}], loss: {loss.item():.4f}")

    # Validacion
    model.eval()
    valid_steps = len(valid_loader) # nro. de batch
    valid_loss = []
    for i, (images, labels) in enumerate(valid_loader):
      output = model(images)
      loss   = loss_function(output, labels)
      valid_loss.append(loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], valid_loss: {np.mean(valid_loss):.4f}")


# Evaluación del modelo

La evualición es un paso fundamental para medir el rendimiento del modelo en datos no vistos durante el entrenamiento y ajustar hiperparámetros.

El conjunto de validación se utiliza para seleccionar el mejor modelo y ajustar hiperparámetros como la tasa de aprendizaje, la regularización, y la arquitectura del modelo. **No se utiliza para entrenar el modelo**; en cambio, se evalúa periódicamente durante el entrenamiento para verificar si el modelo está mejorando y para prevenir el sobreajuste.


In [ ]:
num_epochs=1

train(model, train_data, valid_data, optimizer, num_epochs)

In [ ]:
def get_accuracty(outputs, labels):
  preds  = np.argmax(outputs.detach().numpy(), axis=1)
  labels = labels.numpy()

  acc = (preds == labels)

  return np.mean(acc)

def evaluacion_test_data(model, test_loader):
  model.eval()

  loss_it = []
  acc_it  = []

  total_steps = len(test_loader)

  for i, (images, labels) in enumerate(test_loader):
    outputs = model(images)

    loss = loss_function(outputs, labels)
    acc  = get_accuracty(outputs, labels)

    loss_it.append(loss.item())
    acc_it.append(acc.item())

  print('Loss={:.4f}, Accuracy={:.4f}'.format(np.mean(loss_it), np.mean(acc_it)))

In [ ]:
evaluacion_test_data(model, test_loader)

In [ ]:
figure = plt.figure(figsize=(8, 9))
cols, rows = 5, 5

model.eval()

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(test_data), size=(1,)).item()
    image, label = test_data[sample_idx]

    output = model(image.unsqueeze(0))

    pred = np.argmax(output.detach().numpy(), axis=1)[0]
    figure.add_subplot(rows, cols, i)
    plt.title(pred)
    plt.axis("off")
    plt.imshow(image.squeeze(), cmap="inferno")
plt.show()